wikipedia情報からキャラクターの情報を構造化してまとめたjsonファイルを作成するソースコード

In [1]:
!pip install wikipedia

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11707 sha256=88250110ef938f4b70d1d3533d5f77a4073ee72f6426dc2d2e5f9a40b06bb0cc
  Stored in directory: c:\users\itohi\appdata\local\pip\cache\wheels\5e\b6\c5\93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [8]:
import wikipedia

wikipedia.set_lang("ja")
res = wikipedia.search("ベートーベン")

wp = wikipedia.page(res[0])
wp_content = wp.content

In [26]:
import re

def extract_dictionary(text):
    # パターン: '== キー ==' の形式を見つける
    pattern = r'={2,3}\s*(.*?)\s*={2,3}'
    
    # キーの位置を見つける
    keys = re.findall(pattern, text)
    
    # キーの位置を基に、次のキーの直前までをバリューとして抽出
    positions = [match.span() for match in re.finditer(pattern, text)]
    values = [text[positions[i][1]:positions[i + 1][0] if i + 1 < len(positions) else None].strip() for i in range(len(keys))]

    if (len(values) == 0 or values[0] == ""):
        return ""
    
    # print(values)

    # 辞書に格納
    # return values
    return dict(zip(keys, values))



In [194]:
import re

def extract_hierarchy(texts):
    hierarchy = {}
    current_parent = None
    current_subsection = None

    for text in texts:
        parent_match = re.search(r'^==\s(.*?)\s==', text)
        subsec_match = re.search(r'^===\s(.*?)\s===', text)

        if parent_match:
            # 親コンテンツを処理
            current_parent = parent_match.group(1)
            current_subsection = None
            parent_content = text[parent_match.end()+1:] if parent_match.end() is not None else ''
            
            hierarchy[current_parent] = {'content': parent_content, 'subsections': {}}
        elif subsec_match and current_parent:
            # サブコンテンツを処理
            current_subsection = subsec_match.group(1)
            subsection_content = text[subsec_match.end()+1:] if subsec_match.end() is not None else ''
            hierarchy[current_parent]['subsections'][current_subsection] = subsection_content

    return hierarchy


In [195]:
wp_content_list = wp_content.split("\n\n\n")

wp_content_list_after_scrutiny = []

for content in wp_content_list:
    wp_content_list_after_scrutiny.append(content)

content_dict = extract_hierarchy(wp_content_list_after_scrutiny)


In [193]:
import json

with open("wiki.json","w") as f:
    json.dump(content_dict,f,ensure_ascii=False)